In [4]:
n

ImportError: cannot import name 'HubertForSpeechClassification' from 'transformers' (/home/mk8574/.conda/envs/mk8574_3.10/lib/python3.10/site-packages/transformers/__init__.py)

In [5]:
import transformers

In [6]:
from transformers import pipeline

pipe = pipeline("audio-classification", model="Rajaram1996/Hubert_emotion")

Some weights of HubertForSequenceClassification were not initialized from the model checkpoint at Rajaram1996/Hubert_emotion and are newly initialized: ['projector.weight', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original1', 'projector.bias', 'classifier.weight', 'classifier.bias', 'hubert.encoder.pos_conv_embed.conv.parametrizations.weight.original0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [17]:
from transformers import AutoProcessor

processor = AutoProcessor.from_pretrained("Rajaram1996/Hubert_emotion")
#model = HubertForSpeechClassification.from_pretrained("Rajaram1996/Hubert_emotion")

OSError: Can't load tokenizer for 'Rajaram1996/Hubert_emotion'. If you were trying to load it from 'https://huggingface.co/models', make sure you don't have a local directory with the same name. Otherwise, make sure 'Rajaram1996/Hubert_emotion' is the correct path to a directory containing all relevant files for a Wav2Vec2CTCTokenizer tokenizer.

In [13]:
from transformers import HubertModel, HubertConfig

# Initializing a Hubert facebook/hubert-base-ls960 style configuration
configuration = HubertConfig()

# Initializing a model from the facebook/hubert-base-ls960 style configuration
model = HubertModel(configuration)

# Accessing the model configuration
configuration = model.config

In [14]:
configuration

HubertConfig {
  "activation_dropout": 0.1,
  "apply_spec_augment": true,
  "attention_dropout": 0.1,
  "bos_token_id": 1,
  "classifier_proj_size": 256,
  "conv_bias": false,
  "conv_dim": [
    512,
    512,
    512,
    512,
    512,
    512,
    512
  ],
  "conv_kernel": [
    10,
    3,
    3,
    3,
    3,
    2,
    2
  ],
  "conv_stride": [
    5,
    2,
    2,
    2,
    2,
    2,
    2
  ],
  "ctc_loss_reduction": "sum",
  "ctc_zero_infinity": false,
  "do_stable_layer_norm": false,
  "eos_token_id": 2,
  "feat_extract_activation": "gelu",
  "feat_extract_norm": "group",
  "feat_proj_dropout": 0.0,
  "feat_proj_layer_norm": true,
  "final_dropout": 0.1,
  "hidden_act": "gelu",
  "hidden_dropout": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "layerdrop": 0.1,
  "mask_feature_length": 10,
  "mask_feature_min_masks": 0,
  "mask_feature_prob": 0.0,
  "mask_time_length": 10,
  "mask_time_min_masks": 2,
  "mask_ti

In [71]:
train_path = '/scratch/network/mk8574/audio_sentiment_challenge/data/train'

In [26]:
emo_dict ={
"angry":0,
 "fear":1,
"sad":2,
 "disgust":3,
 "neutral":4,
"happy" :5,
"surprise":0
}

In [36]:
for PATH in tqdm(glob(os.path.join(train_path, '*.wav'))):
    print(PATH)
    break

  0%|          | 0/5001 [00:00<?, ?it/s]

/scratch/network/mk8574/audio_sentiment_challenge/data/train/TRAIN_0000.wav


In [39]:
test_path = '/scratch/network/mk8574/audio_sentiment_challenge/data/test'

In [72]:
from glob import glob
import os
from tqdm import tqdm
preds=[]
preds2=[]
for PATH in tqdm(glob(os.path.join(train_path, '*.wav'))):
    d=pipe(PATH,top_k=14)
    s=[0]*6
    
    for i in d:
        s[emo_dict[i["label"].split("_")[1]]]+=i["score"]
    preds2.append(s.index(max(s)))
    
    preds.append(emo_dict[pipe(PATH)[0]["label"].split("_")[1]])


100%|██████████| 5001/5001 [11:02<00:00,  7.55it/s]


In [45]:
import pandas as pd
working_dir = '/scratch/network/mk8574/audio_sentiment_challenge'
submission = pd.read_csv(os.path.join(working_dir, 'baseline_dy', 'sample_submission.csv'))
submission['label'] = preds
submission.to_csv(os.path.join(working_dir, 'baseline_dy', 'pretrained_submission.csv'), index=False)

In [73]:
import pandas as pd
import os
working_dir = '/scratch/network/mk8574/audio_sentiment_challenge'
train_csv = pd.read_csv(os.path.join(working_dir, 'data', 'train.csv'))
train_pred = train_csv['label']
print(sum(1 for x,y in zip(train_pred,preds) if x == y) / len(preds))# just
print(sum(1 for x,y in zip(train_pred,preds2) if x == y) / len(preds2))# just

0.21055788842231554
0.17336532693461307
